In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler


# Setting to make numbers easier to read on display
pd.options.display.float_format = "{:20.2f}".format

# Show all columns on output
pd.set_option("display.max_columns", 999)

# Data Exploration

In [ ]:
df = pd.read_csv("online_retail_II.csv")
df.head(10)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include=[object])

In [ ]:
df[df["Customer ID"].isna()].head(10)

In [ ]:
print(df.shape)

In [ ]:
df[df["Quantity"] < 0].head(4)

In [ ]:
print(df[df["Quantity"] < 0 ].shape)

In [ ]:
df.columns

In [ ]:
df["Invoice"] = df["Invoice"].astype(str)
df[df["Invoice"].str.match("^\\d{6}$") == True]

In [ ]:
df[df["Invoice"].str.match("^\\d{6}$") == False]

In [ ]:
df["Invoice"].str.replace("[0-9]","",regex = True).unique()

In [ ]:
df[df["Invoice"].str.startswith("A")]

In [ ]:
df["StockCode"] = df["StockCode"].astype("str")
df[df["StockCode"].str.match("^\\d{5}$") == True]

In [ ]:
df[df["StockCode"].str.match("^\\d{5}$") == False]

In [ ]:
df[(df["StockCode"].str.match("^\\d{5}$") == False ) & (df["StockCode"].str.match("^\\d{5}[a-zA-Z]+$") == False)]["StockCode"].unique()

In [ ]:
df[df["StockCode"].str.contains("^CRUK")]

# Stock Code
### . StockCode is meant to follow the pattern [0-9]{5} but seens to have legit values for [0-9]{5}[a-zA-Z]+

#### Also contians other values
| Code              | Description | Action |
| :---------------- | :------ | ----: |
| DCGS              |  Looks valid, some quantities are negative though and customer ID is null    | Exclude from clustering |
| D                 | Looks valid, represents discount values   | Exclude from clustering |
| DOT               | Looks valid, represents postage charge  | Exclude from clustering |
| M or m            | Looks valid, represents manual transactions   | Exclude from clustering|
| C2                | Carriage transaction - not sure what this means|Exclude from clustering|
| C3|Not sure, only 1 transaction| Exclude |
| BANK CHARGES or B| BAnk charges| Exclude |
|S| Samples sent to customer | EXclude from clustering |
|TESTXXX| Testing data, not valid|Exclude|
|gift_XXX|Purchase wit gift cards, might be interesting for another analysis, but no customer data| Exclude|
|PADS|Looks like aslegit stock code for padding|Include|
|SP1002|Looks like a special request item, only 2 transactions, 3 look legit, 1 has 0 pricing| Exclude for now|
|AMAZONFEE|Looks like fees for Amazon shipping or something|EXclude for now|
|ADJUST|Looks like manual account adjustments by admins|Exclude|


In [ ]:
df[df["StockCode"].str.contains("^PADS")]

In [ ]:
df[df["StockCode"].str.contains("^BANK CHARGES")]

In [ ]:
 df[(df["StockCode"].str.match("^\\d{5}$") == False ) & (df["StockCode"].str.match("^\\d{5}[a-zA-Z]+$") == False)]["StockCode"].unique()

# Data Cleaning

In [ ]:
cleaned_df = df.copy()
cleaned_df

In [ ]:
cleaned_df["Invoice"] = cleaned_df["Invoice"].astype("str")
mask = (
    cleaned_df["Invoice"].str.match("^\\d{6}$") == True
)
cleaned_df = cleaned_df[mask]
cleaned_df

In [ ]:
cleaned_df["StockCode"] = cleaned_df["StockCode"].astype("str")

mask = (
    (cleaned_df["StockCode"].str.match("^\\d{5}$")== True)
    |(cleaned_df["StockCode"].str.match("^\\d{5}[a-zA-Z]+$")== True)
    |(cleaned_df["StockCode"].str.match("^PADS$")== True)
)

cleaned_df = cleaned_df[mask]
cleaned_df

In [ ]:
cleaned_df.describe()

In [ ]:
cleaned_df.dropna(subset=["Customer ID"], inplace=True)

In [ ]:
cleaned_df.describe()

In [ ]:
len(cleaned_df[cleaned_df["Price"] == 0])

In [ ]:
cleaned_df = cleaned_df[cleaned_df["Price"] > 0.0]

In [ ]:
cleaned_df.describe()

In [ ]:
cleaned_df["Price"].min()

In [ ]:
len(cleaned_df) / len(df)

### Droppped about 25% data during data cleaning process

# Feature Engineering

In [ ]:
cleaned_df["SalesLineTotal"] = cleaned_df["Quantity"] * cleaned_df["Price"]

In [ ]:
cleaned_df.head(2)

In [ ]:
aggregated_df = cleaned_df.groupby(by="Customer ID", as_index=False) \
.agg(
     MonetaryValue = ("SalesLineTotal", "sum"),
     Frequency = ("Invoice", "nunique"),
     LastInvoiceDate = ("InvoiceDate", "max")
)

In [ ]:
aggregated_df["LastInvoiceDate"] = pd.to_datetime(aggregated_df["LastInvoiceDate"])

In [ ]:
max_invoice_date = aggregated_df["LastInvoiceDate"].max()

In [ ]:
max_invoice_date

In [ ]:
aggregated_df["Recency"] = (max_invoice_date - aggregated_df["LastInvoiceDate"]).dt.days

In [ ]:
aggregated_df.head(4)

### Distribution Plot

In [ ]:
plt.figure(figsize=(15,5))

# Histogram for Monetary value
plt.subplot(1,3,1)
plt.hist(aggregated_df["MonetaryValue"], bins = 10, color = "skyblue", edgecolor = "black")
plt.title("Monetary Value distribution")
plt.xlabel("Monetary Value")
plt.ylabel("Count")

# Histrogram for Frequecny
plt.subplot(1,3,2)
plt.hist(aggregated_df["Frequency"], bins = 10, color = "lightgreen", edgecolor = "black")
plt.title("Frequency distribution")
plt.xlabel("Frequnecy")
plt.ylabel("Count")

# Histogram for Recency
plt.subplot(1,3,3)
plt.hist(aggregated_df["Recency"], bins = 20, color = "salmon", edgecolor = "black")
plt.title("Recency distribution")
plt.xlabel("Recency Value")
plt.ylabel("Count")

plt.tight_layout()
plt.show()

### Boxplot with Outliers

In [ ]:
plt.figure(figsize=(15,5))

# Boxplot for Monetary value
plt.subplot(1,3,1)
sns.boxplot(data = aggregated_df["MonetaryValue"],color="skyblue")
plt.title("Monetary Value Boxplot")
plt.xlabel("Monetary Value")

# Boxoplot for Frequecny
plt.subplot(1,3,2)
sns.boxplot(data = aggregated_df["Frequency"], color = "lightgreen")
plt.title("Frequency Boxplot")
plt.xlabel("Frequnecy")

# Boxplot for Recency
plt.subplot(1,3,3)
sns.boxplot(data = aggregated_df["Recency"],color = "salmon")
plt.title("Recency Boxplot")
plt.xlabel("Recency")

plt.tight_layout()
plt.show()

# Removing Outliers

### From Monetary Value

In [ ]:
M_Q1 = aggregated_df["MonetaryValue"].quantile(0.25)
M_Q3 = aggregated_df["MonetaryValue"].quantile(0.75)
M_IQR = M_Q3 - M_Q1

monetary_outliers_df = aggregated_df[(aggregated_df["MonetaryValue"] > (M_Q3 + 1.5 * M_IQR)) | (aggregated_df["MonetaryValue"] < (M_Q1 - 1.5 * M_IQR))].copy()

monetary_outliers_df.describe()

### From Frequency

In [ ]:
F_Q1 = aggregated_df["Frequency"].quantile(0.25)
F_Q3 = aggregated_df["Frequency"].quantile(0.75)
F_IQR = F_Q3 - F_Q1

frequency_outliers_df = aggregated_df[(aggregated_df["Frequency"] > (F_Q3 + 1.5 * F_IQR )) | (aggregated_df["Frequency"] < (F_Q1 - 1.5 * F_IQR))]
frequency_outliers_df.describe()

In [ ]:
non_outliers_df = aggregated_df[(~aggregated_df.index.isin(monetary_outliers_df.index)) & (~aggregated_df.index.isin(frequency_outliers_df.index))]
non_outliers_df.describe()

### Boxplot  without outlier

In [ ]:
plt.figure(figsize=(15,5))

# Boxplot for monetary value
plt.subplot(1, 3, 1)
sns.boxplot(data = non_outliers_df["MonetaryValue"], color="skyblue")
plt.title("Monetary Value Boxplot")
plt.xlabel("Monetary Value")

# Boxplot for frequency
plt.subplot(1,3,2)
sns.boxplot(data = non_outliers_df["Frequency"], color="lightgreen")
plt.title("Frequency Boxplot")
plt.xlabel("Frequncy")

# Boxplot for recency
plt.subplot(1,3,3)
sns.boxplot(data = non_outliers_df["Recency"], color="salmon")
plt.title("Recency Boxplot")
plt.xlabel("Recency")

plt.tight_layout()
plt.show()


In [ ]:
fig = plt.figure(figsize=(8,8))

ax = fig.add_subplot(projection="3d")

scatter = ax.scatter(non_outliers_df["MonetaryValue"], non_outliers_df["Frequency"], non_outliers_df["Recency"])

ax.set_xlabel("Monetary Value")
ax.set_ylabel("Frequency")
ax.set_zlabel("Recency")

ax.set_title("3D Scatter Plot of Customer Data")

plt.show()

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(non_outliers_df[["MonetaryValue", "Frequency", "Recency"]])
scaled_data

In [ ]:
scaled_data_df = pd.DataFrame(scaled_data, index=non_outliers_df.index, columns= ["MonetaryValue", "Frequency", "Recency"])
scaled_data_df.head()

In [ ]:
fig = plt.figure(figsize=(8,8))

ax = fig.add_subplot(projection="3d")

scatter = ax.scatter(scaled_data_df["MonetaryValue"], scaled_data_df["Frequency"], scaled_data_df["Recency"])

ax.set_xlabel("Monetary Value")
ax.set_ylabel("Frequency")
ax.set_zlabel("Recency")

ax.set_title("3D Scatter Plot of Customer Data")

plt.show()

# KMeans Clustering

### K value through elbow method and silhouette score

In [ ]:
max_k = 12

inertia = []
silhouette_scores = []
k_values = range(2, max_k+1)

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, max_iter=1000)
    
    cluster_labels = kmeans.fit_predict(scaled_data_df)
    
    sil_score = silhouette_score(scaled_data_df, cluster_labels)
    
    silhouette_scores.append(sil_score)
    
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(14,6))

plt.subplot(1,2,1)
plt.plot(k_values, inertia, marker = "o")
plt.title("KMeans Inertia for Different Values of k")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Inertia")
plt.xticks(k_values)
plt.grid(True)

plt.subplot(1,2,2)
plt.plot(k_values, silhouette_scores, marker = "o", color = "orange")
plt.title("Silhouette scores for Different values of k")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Silhouette Score")
plt.xticks(k_values)
plt.grid(True)

plt.tight_layout()
plt.show()

# Final Model

In [ ]:
kmeans = KMeans(n_clusters=4,random_state=42, max_iter=1000 )

cluster_labels = kmeans.fit_predict(scaled_data_df)

cluster_labels

In [ ]:
non_outliers_df["Cluster"] = cluster_labels

non_outliers_df

In [ ]:
cluster_colors = {0:"Blue",
                  1:"Orange",
                  2:"Green",
                  3:"Red"}

colors = non_outliers_df["Cluster"].map(cluster_colors)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(projection="3d")

scatter = ax.scatter(non_outliers_df["MonetaryValue"],
                     non_outliers_df["Frequency"],
                     non_outliers_df["Recency"],
                     c = colors,  # used mapped solid colors
                     marker = "o")


ax.set_xlabel("Monetary Value")
ax.set_ylabel("Frequency")
ax.set_zlabel("Recency")

ax.set_title("3D Scatter Plot of Customer Data by Cluster")

plt.show()


## Violin Plot 

In [ ]:
plt.figure(figsize=(12,18))

# for Monetary Value
plt.subplot(3,1,1)
sns.violinplot(x = non_outliers_df["Cluster"], y = non_outliers_df["MonetaryValue"], palette=cluster_colors,hue=non_outliers_df["Cluster"])
sns.violinplot(y = non_outliers_df["MonetaryValue"], color='gray', linewidth=1.0)
plt.title("Monetary Value by Cluster")
plt.ylabel("Monetary value")

# for Frequency
plt.subplot(3,1,2)
sns.violinplot(x = non_outliers_df["Cluster"], y = non_outliers_df["Frequency"], palette=cluster_colors, hue = non_outliers_df["Cluster"])
sns.violinplot(y = non_outliers_df["Frequency"], color='gray', linewidth=1.0)
plt.title("Frequency by Cluster")
plt.ylabel("Frequency")

# for Recency
plt.subplot(3,1,3)
sns.violinplot(x = non_outliers_df["Cluster"], y = non_outliers_df["Recency"], palette=cluster_colors, hue =non_outliers_df["Cluster"])
sns.violinplot(y = non_outliers_df["Recency"], color='gray', linewidth=1.0)
plt.ylabel("Recency")

plt.tight_layout()
plt.show()

### 1. Cluster 0 (Blue): "Reward"

##### . Rationale: This cluster represents high value frequent buyers many of whom are still actively purchasing.They are your most loyal customers, and rewarding their loyalty is key to maintaining their engagement.
##### . Action: Implement a robust loyalty program, provide exclusive offers and recognize their loyalty to keep them engaged and satisfied.

### 2. Cluster 1 (Orange): "Re-Engage"

##### . Rationale: This group includes lower-value, infrequent buyers who haven't purchased recently. The focus should be on re-engagement to bring them back into active purchaising behavior.
##### . Action: Use targeted marketing campaigns, special discounts,or reminders to encourage them to return and purchased.

### 3. Cluster 2 (Green): "Retain"

##### . Rationale: This cluster represents high value customers who purchase regularly, though not always very recently. The focus should be on retention efforts to maintain their loyalty and spending levels.
##### . Action: Implement loyalty programs, personalized offers, and regular engagement to ensure they remain active.

### 4. Cluster 3 (Red): "Nurture"

##### . Rationale: This cluster represents the least active and lowest-value customers, but they have made recent purchases.These customers may be new or need nurturing to increase their enagagement and spending.
##### . Action: Focus on building relationships,providing excellent customer service and offering incentives to encourage more frequent purchases.

### Summary of Clsuter Names:

#### . Cluster 0(Blue):"Reward"
#### . Cluster 1(Orange):"Re-Engage"
#### . Cluster 2(Green):"Retain"
#### . Cluster 3(Red):"Nurture"
 

In [ ]:
overlap_indices = monetary_outliers_df.index.intersection(frequency_outliers_df.index)

monetary_only_outliers = monetary_outliers_df.drop(overlap_indices)
frequency_only_outliers = frequency_outliers_df.drop(overlap_indices)

monetary_and_frequency_outliers = monetary_outliers_df.loc[overlap_indices]

In [ ]:
monetary_only_outliers["Cluster"] = -1
frequency_only_outliers["Cluster"] = -2
monetary_and_frequency_outliers["Cluster"] = -3

outlier_clsuters_df = pd.concat([monetary_only_outliers, frequency_only_outliers, monetary_and_frequency_outliers])
outlier_clsuters_df

In [ ]:
cluster_colors = {-1:'#9467bd',
                  -2:'#8c564b',
                  -3:'#e377c2'}

plt.figure(figsize=(12,18))

plt.figure(figsize=(12,18))

# for Monetary Value
plt.subplot(3,1,1)
sns.violinplot(x = outlier_clsuters_df["Cluster"], y = outlier_clsuters_df["MonetaryValue"], palette=cluster_colors,hue=outlier_clsuters_df["Cluster"])
sns.violinplot(y = non_outliers_df["MonetaryValue"], color='gray', linewidth=1.0)
plt.title("Monetary Value by Cluster")
plt.ylabel("Monetary value")

# for Frequency
plt.subplot(3,1,2)
sns.violinplot(x = outlier_clsuters_df["Cluster"], y = outlier_clsuters_df["Frequency"], palette=cluster_colors, hue = outlier_clsuters_df["Cluster"])
sns.violinplot(y = outlier_clsuters_df["Frequency"], color='gray', linewidth=1.0)
plt.title("Frequency by Cluster")
plt.ylabel("Frequency")

# for Recency
plt.subplot(3,1,3)
sns.violinplot(x = outlier_clsuters_df["Cluster"], y = outlier_clsuters_df["Recency"], palette=cluster_colors, hue =outlier_clsuters_df["Cluster"])
sns.violinplot(y = outlier_clsuters_df["Recency"], color='gray', linewidth=1.0)
plt.ylabel("Recency")

plt.tight_layout()
plt.show()


#### Cluster -1(Monetary Outliers) PAMPER: 
##### Characteristics: High spenders but not necessarily frequent buyers. Their purchases are large but infrequent . 
##### Potential Strategy: Focus on maintaining their loyalty with personalized offers or luxury services that cater to their high spending capacity.

#### Cluster -2(Frequency Outliers) UPSELL: 
##### Characteristics: Frequent buyers who spend less per purchase.Thses customers are consistently engaged but might benefit from upselling opportunities . 
##### Potential Strategy: Implement loyalty programs  or bundle deals to encourage higher spending per visit, given their frequent engagement.

#### Cluster -3(Monetary & Frequency Outliers) DELIGHT: 
##### Characteristics: The most valuable outliers  with extreme spending and frequent purchases.They are likely your top-tier customers who require special attention. 
##### Potential Strategy: Develop VIP programs or exclusive offers to maintain their loyalty and encourage continued engagemnet.

In [ ]:
cluster_labels = {
     0 : "REWARD",
     1 : "RE-ENGAGE",
     2 : "RETAIN",
     3 : "NURTURE",
    -1 : "PAMPER",
    -2 : "UPSELL",
    -3 : "DELIGHT",
}

In [ ]:
full_clustering_df = pd.concat([non_outliers_df, outlier_clsuters_df])
full_clustering_df

In [ ]:
full_clustering_df["ClusterLabel"] = full_clustering_df["Cluster"].map(cluster_labels)

full_clustering_df

# Visualization

In [ ]:
cluster_counts = full_clustering_df["ClusterLabel"].value_counts()
full_clustering_df["MonetaryValue per 100 pounds"] = full_clustering_df["MonetaryValue"] / 100.00
feature_means = full_clustering_df.groupby("ClusterLabel")[["Recency", "Frequency", "MonetaryValue per 100 pounds"]].mean()

fig, ax1 = plt.subplots(figsize = (12,8))

sns.barplot(x = cluster_counts.index, y = cluster_counts.values, ax = ax1, palette="viridis", hue = cluster_counts.index)
ax1.set_ylabel("Number of Customers", color = "b")
ax1.set_title("Cluster Distribution with Average Feature Values")

ax2 = ax1.twinx()

sns.lineplot(data = feature_means, ax = ax2, palette="Set2", marker = "o")
ax2.set_ylabel("Average Value", color = "g")

plt.show()